# DEEP MLP ARCHITECTURES

In [17]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
from scipy.sparse import csr_matrix 
from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss

# if you keras is not using tensorflow as backend set "KERAS_BACKEND=tensorflow" use this command
from keras.utils import np_utils 
from keras.datasets import mnist 
import seaborn as sns
from keras.initializers import RandomNormal
import tensorflow as tf
from keras.layers.normalization import BatchNormalization

from keras.models import Sequential 
from keras.layers import Dense, Activation 
from keras.layers import Dropout

In [2]:
df_train = pd.read_csv('E:/BOOKS_NEW/Cases datasets/1st/reducing-commercial-aviation-fatalities/train_after_smoothening_2.csv')

In [3]:
import re
df_train = df_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [4]:
train, test = train_test_split(df_train, test_size=0.2, random_state=42, shuffle=True)

In [5]:
x_train = train.loc[:, df_train.columns != 'event']
y_train = train['event']

x_test = test.loc[:, df_train.columns != 'event']
y_test = test['event']

print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)
print('-----------------------------------------------')
print('Features of x_train : ', x_train.columns.values)
print('Features of x_test: ', x_test.columns.values)
print('-----------------------------------------------')
print('Values of x_train : ', x_train.values)
print('Values of x_test : ', x_test.values)
print('-----------------------------------------------')
print('Values of y_train: ', y_train.values)
print('Values of y_test: ', y_test.values)

(3893936, 31) (3893936,)
(973485, 31) (973485,)
-----------------------------------------------
Features of x_train :  ['Unnamed0' 'Unnamed01' 'Unnamed011' 'Unnamed0111' 'crew' 'experiment'
 'time' 'seat' 'smoothened_ecg_data' 'smoothened_r_data'
 'smoothened_gsr_data' 'smoothened_eeg_fp1' 'smoothened_eeg_f7'
 'smoothened_eeg_f8' 'smoothened_eeg_t4' 'smoothened_eeg_t6'
 'smoothened_eeg_t5' 'smoothened_eeg_t3' 'smoothened_eeg_fp2'
 'smoothened_eeg_o1' 'smoothened_eeg_p3' 'smoothened_eeg_pz'
 'smoothened_eeg_f3' 'smoothened_eeg_fz' 'smoothened_eeg_f4'
 'smoothened_eeg_c4' 'smoothened_eeg_p4' 'smoothened_eeg_poz'
 'smoothened_eeg_c3' 'smoothened_eeg_cz' 'smoothened_eeg_o2']
Features of x_test:  ['Unnamed0' 'Unnamed01' 'Unnamed011' 'Unnamed0111' 'crew' 'experiment'
 'time' 'seat' 'smoothened_ecg_data' 'smoothened_r_data'
 'smoothened_gsr_data' 'smoothened_eeg_fp1' 'smoothened_eeg_f7'
 'smoothened_eeg_f8' 'smoothened_eeg_t4' 'smoothened_eeg_t6'
 'smoothened_eeg_t5' 'smoothened_eeg_t3' 'smoo

In [8]:
#define min max scaler
scaler = MinMaxScaler()
#transform data
x_train = scaler.fit_transform(x_train)
print(x_train)
print('------------------------------')
x_test = scaler.fit_transform(x_test)
print(x_test)

[[0.06755736 0.06755736 0.06755736 ... 0.54502693 0.49846334 0.48371547]
 [0.61251004 0.61251004 0.61251004 ... 0.52741097 0.46480423 0.47597925]
 [0.13371707 0.13371707 0.13371707 ... 0.52999312 0.47212553 0.48051439]
 ...
 [0.4590704  0.4590704  0.4590704  ... 0.53107181 0.47439914 0.47825508]
 [0.88436418 0.88436418 0.88436418 ... 0.53846916 0.47227077 0.47964164]
 [0.34776994 0.34776994 0.34776994 ... 0.53773068 0.48333051 0.47873265]]
------------------------------
[[0.90810665 0.90810665 0.90810665 ... 0.57036781 0.50367042 0.39534804]
 [0.77045409 0.77045409 0.77045409 ... 0.5732391  0.50335701 0.40077842]
 [0.57436392 0.57436392 0.57436392 ... 0.5800763  0.51326306 0.39658845]
 ...
 [0.63132004 0.63132004 0.63132004 ... 0.60220402 0.5115242  0.37669904]
 [0.36971193 0.36971193 0.36971193 ... 0.56773574 0.50973704 0.40625457]
 [0.67497608 0.67497608 0.67497608 ... 0.57468218 0.50749457 0.3949462 ]]


In [9]:
x_train.shape[0], x_train.shape[1]

(3893936, 31)

In [10]:
x_test.shape[0], x_test.shape[1]

(973485, 31)

In [14]:
output_dim = 4
input_dim = x_train.shape[1]
print(input_dim)
batch_size = 100000 
nb_epoch = 20

31


# Model 1

In [15]:
model_relu = Sequential()
model_relu.add(Dense(64, activation='relu', input_shape=(input_dim,), kernel_initializer=RandomNormal(mean=0.0, stddev=0.062, seed=None)))
model_relu.add(Dense(32, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.125, seed=None)) )
model_relu.add(Dense(output_dim, activation='softmax'))

print(model_relu.summary())

model_relu.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model_relu.fit(x_train, y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(x_test, y_test))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                2048      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 132       
Total params: 4,260
Trainable params: 4,260
Non-trainable params: 0
_________________________________________________________________
None

Train on 3893936 samples, validate on 973485 samples
Epoch 1/20
3893936/3893936 [==============================] - 6s 1us/step - loss: 1.0560 - accuracy: 0.5902 - val_loss: 0.8438 - val_accuracy: 0.5852
Epoch 2/20
3893936/3893936 [==============================] - 5s 1us/step - loss: 0.7781 - accuracy: 0.7071 - val_loss: 0.6926 - val_accuracy: 0.8721
Epoch 3/20
3893936/3893

# MODEL 2(MLP+Dropout+ADAM)

In [18]:
model_drop = Sequential()

model_drop.add(Dense(551, activation='relu', input_shape=(input_dim,), kernel_initializer=RandomNormal(mean=0.0, stddev=0.039, seed=None)))
model_drop.add(BatchNormalization())
model_drop.add(Dropout(0.5))

model_drop.add(Dense(447, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.55, seed=None)) )
model_drop.add(BatchNormalization())
model_drop.add(Dropout(0.5))

model_drop.add(Dense(331, activation='relu', input_shape=(input_dim,), kernel_initializer=RandomNormal(mean=0.0, stddev=0.039, seed=None)))
model_drop.add(BatchNormalization())
model_drop.add(Dropout(0.5))

model_drop.add(Dense(236, activation='relu', input_shape=(input_dim,), kernel_initializer=RandomNormal(mean=0.0, stddev=0.039, seed=None)))
model_drop.add(BatchNormalization())
model_drop.add(Dropout(0.5))

model_drop.add(Dense(121, activation='relu', input_shape=(input_dim,), kernel_initializer=RandomNormal(mean=0.0, stddev=0.039, seed=None)))
model_drop.add(BatchNormalization())
model_drop.add(Dropout(0.5))

model_drop.add(Dense(output_dim, activation='softmax'))


model_drop.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 551)               17632     
_________________________________________________________________
batch_normalization_2 (Batch (None, 551)               2204      
_________________________________________________________________
dropout_1 (Dropout)          (None, 551)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 447)               246744    
_________________________________________________________________
batch_normalization_3 (Batch (None, 447)               1788      
_________________________________________________________________
dropout_2 (Dropout)          (None, 447)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 331)              

In [19]:
model_drop.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model_drop.fit(x_train, y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(x_test, y_test))

Train on 3893936 samples, validate on 973485 samples
Epoch 1/20
3893936/3893936 [==============================] - 275s 71us/step - loss: 0.5036 - accuracy: 0.8441 - val_loss: 1.9103 - val_accuracy: 0.5852
Epoch 2/20
3893936/3893936 [==============================] - 273s 70us/step - loss: 0.2837 - accuracy: 0.9164 - val_loss: 2.6898 - val_accuracy: 0.5852
Epoch 3/20
3893936/3893936 [==============================] - 271s 70us/step - loss: 0.2693 - accuracy: 0.9204 - val_loss: 3.1034 - val_accuracy: 0.5852
Epoch 4/20
3893936/3893936 [==============================] - 271s 70us/step - loss: 0.2614 - accuracy: 0.9222 - val_loss: 3.2574 - val_accuracy: 0.5852
Epoch 5/20
3893936/3893936 [==============================] - 272s 70us/step - loss: 0.2565 - accuracy: 0.9230 - val_loss: 3.2561 - val_accuracy: 0.5852
Epoch 6/20
3893936/3893936 [==============================] - 271s 70us/step - loss: 0.2533 - accuracy: 0.9233 - val_loss: 2.9330 - val_accuracy: 0.5852
Epoch 7/20
3893936/3893936 [=

# MODEL 3 (MLP+ReLu+ADAM)

In [20]:
model_relu = Sequential()
model_relu.add(Dense(551, activation='relu', input_shape=(input_dim,), kernel_initializer=RandomNormal(mean=0.0, stddev=0.062, seed=None)))
model_relu.add(Dense(447, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.125, seed=None)) )
model_relu.add(Dense(331, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.125, seed=None)) )
model_relu.add(Dense(236, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.125, seed=None)) )
model_relu.add(Dense(121, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.125, seed=None)) )
model_relu.add(Dense(output_dim, activation='softmax'))

print(model_relu.summary())

model_relu.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model_relu.fit(x_train, y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(x_test, y_test))

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 551)               17632     
_________________________________________________________________
dense_15 (Dense)             (None, 447)               246744    
_________________________________________________________________
dense_16 (Dense)             (None, 331)               148288    
_________________________________________________________________
dense_17 (Dense)             (None, 236)               78352     
_________________________________________________________________
dense_18 (Dense)             (None, 121)               28677     
_________________________________________________________________
dense_19 (Dense)             (None, 4)                 488       
Total params: 520,181
Trainable params: 520,181
Non-trainable params: 0
________________________________________________

# MODEL 4 (MLP+Dropout+ADAM)

In [21]:
model_drop = Sequential()

model_drop.add(Dense(438, activation='relu', input_shape=(input_dim,), kernel_initializer=RandomNormal(mean=0.0, stddev=0.039, seed=None)))
model_drop.add(BatchNormalization())
model_drop.add(Dropout(0.5))

model_drop.add(Dense(276, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.55, seed=None)) )
model_drop.add(BatchNormalization())
model_drop.add(Dropout(0.5))

model_drop.add(Dense(157, activation='relu', input_shape=(input_dim,), kernel_initializer=RandomNormal(mean=0.0, stddev=0.039, seed=None)))
model_drop.add(BatchNormalization())
model_drop.add(Dropout(0.5))

model_drop.add(Dense(output_dim, activation='softmax'))


model_drop.summary()

model_drop.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model_drop.fit(x_train, y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(x_test, y_test))

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 438)               14016     
_________________________________________________________________
batch_normalization_7 (Batch (None, 438)               1752      
_________________________________________________________________
dropout_6 (Dropout)          (None, 438)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 276)               121164    
_________________________________________________________________
batch_normalization_8 (Batch (None, 276)               1104      
_________________________________________________________________
dropout_7 (Dropout)          (None, 276)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 157)              

# MODEL 5

In [22]:
model_drop = Sequential()

model_drop.add(Dense(612, activation='relu', input_shape=(input_dim,), kernel_initializer=RandomNormal(mean=0.0, stddev=0.039, seed=None)))
model_drop.add(BatchNormalization())
model_drop.add(Dropout(0.5))

model_drop.add(Dense(324, activation='relu', kernel_initializer=RandomNormal(mean=0.0, stddev=0.55, seed=None)) )
model_drop.add(BatchNormalization())
model_drop.add(Dropout(0.5))

model_drop.add(Dense(output_dim, activation='softmax'))


model_drop.summary()

model_drop.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model_drop.fit(x_train, y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, validation_data=(x_test, y_test))

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 612)               19584     
_________________________________________________________________
batch_normalization_10 (Batc (None, 612)               2448      
_________________________________________________________________
dropout_9 (Dropout)          (None, 612)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 324)               198612    
_________________________________________________________________
batch_normalization_11 (Batc (None, 324)               1296      
_________________________________________________________________
dropout_10 (Dropout)         (None, 324)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 4)                

# CONCLUSION

# MODEL 3 GAVE THE BEST RESULT - 
# val_loss: 0.1505 
# val_accuracy: 0.9408

In [4]:
from prettytable import PrettyTable
    
x = PrettyTable()

x.field_names = ["S.NO","MODEL","LOG LOSS"]

x.add_row(["1","MODEL 1: 3 Dense Layers",0.2407])
x.add_row(["2","MODEL 2: 6 Dense Layers with BN and Dropout",1.7042])
x.add_row(["3","MODEL 3: 6 Dense Layers",0.1505])
x.add_row(["4","MODEL 4: 4 Dense Layers with BN and Dropout",0.2389])
x.add_row(["5","MODEL 5: 3 Dense Layers with BN and Dropout",0.2444])

# Printing the Table
print(x)

+------+---------------------------------------------+----------+
| S.NO |                    MODEL                    | LOG LOSS |
+------+---------------------------------------------+----------+
|  1   |           MODEL 1: 3 Dense Layers           |  0.2407  |
|  2   | MODEL 2: 6 Dense Layers with BN and Dropout |  1.7042  |
|  3   |           MODEL 3: 6 Dense Layers           |  0.1505  |
|  4   | MODEL 4: 4 Dense Layers with BN and Dropout |  0.2389  |
|  5   | MODEL 5: 3 Dense Layers with BN and Dropout |  0.2444  |
+------+---------------------------------------------+----------+
